# Training using Multiple GPUs
[Tensorflow](https://rapids.ai/) is a popular, powerful framework for deep learning used by data scientists across industries.

In this example, you'll train a Resnet50 model to identify different species of birds using multiple GPUs. This dataset constitutes 40,000+ birds and has been taken from [kaggle](https://www.kaggle.com/gpiosenka/100-bird-species).


First we import necessary libraries.

In [ ]:
import tensorflow as tf
import keras
import time

print(tf.__version__)

We will be using [Weights & Biases](https://wandb.ai/site) to monitor GPU performance. Users will need to set their own Saturn Cloud environment credential for wandb. Check [here](https://saturncloud.io/docs/examples/python/weights-and-biases/qs-wandb/) on more information on creating and connecting W&B account to Saturn Cloud. First, we log into Weights & Biases. 


In [ ]:
import wandb
from wandb.keras import WandbCallback

wandb.login()

### Extracting Data
The dataset originally had 285 classes. We have taken subset of this data which has 61 classes. The data is stored in AWS S3.The first time you run this job, you'll need to download the training and test data which will be saved in `dataset/birds/`.

In [ ]:
import s3fs

s3 = s3fs.S3FileSystem(anon=True)
_ = s3.get(
    rpath="s3://saturn-public-data/100-bird-species/100-bird-species/*/*/*.jpg",
    lpath="dataset/birds/",
)

Run the code below to ensure that TensorFlow uses only the memory required for the model, instead of using all the GPU RAM it has access to.

In [ ]:
gpus = tf.config.list_physical_devices("GPU")
if gpus:
    try:
        # Currently, memory growth needs to be the same across GPUs
        for gpu in gpus:
            tf.config.experimental.set_memory_growth(gpu, True)
        logical_gpus = tf.config.experimental.list_logical_devices("GPU")
        print(len(gpus), "Physical GPUs,", len(logical_gpus), "Logical GPUs")
    except RuntimeError as e:
        # Memory growth must be set before GPUs have been initialized
        print(e)

### Training 
In code below, we construct Keras data objects for the training and validation sets using `keras.preprocessing.image_dataset_from_directory`. We use the Adam optimizer and set the learning rate to 0.02. 
We will parallelize the learning by using the TensorFlow Mirrored Strategy. The model will split the data in each batch (sometimes called "global batch") across the GPUs, making "worker batches." Each GPU has a copy of the model, called a "replica," and while they learn on different parts of the data, they will combine the learned gradients at the end of the step. They stay synchronized this way, and the result at the end of training is one model that has learned on all the data.

In [ ]:
def train_multigpu(
    n_epochs, classes, base_lr, batchsize, wbargs, scale_batch=False, scale_lr=False
):

    wbargs = {**wbargs, "scale_batch": scale_batch, "scale_lr": scale_lr}

    # --------- Start wandb --------- #
    wandb.init(config=wbargs, project="wandb_saturn_demo")

    strategy = tf.distribute.MirroredStrategy()
    print("Number of devices: %d" % strategy.num_replicas_in_sync)

    if scale_batch:
        batchsize = batchsize * strategy.num_replicas_in_sync

    if scale_lr:
        base_lr = base_lr * strategy.num_replicas_in_sync

    with strategy.scope():
        model = tf.keras.applications.ResNet50(include_top=True, weights=None, classes=classes)

        optimizer = keras.optimizers.Adam(lr=base_lr)
        model.compile(
            loss="sparse_categorical_crossentropy", optimizer=optimizer, metrics=["accuracy"]
        )

    # Data
    train_ds = (
        tf.keras.preprocessing.image_dataset_from_directory(
            "dataset/birds/train", image_size=(224, 224), batch_size=batchsize
        )
        .prefetch(2)
        .cache()
        .shuffle(1000)
    )

    valid_ds = tf.keras.preprocessing.image_dataset_from_directory(
        "dataset/birds/valid", image_size=(224, 224), batch_size=batchsize
    ).prefetch(2)

    start = time.time()

    model.fit(train_ds, epochs=n_epochs, validation_data=valid_ds, callbacks=[WandbCallback()])

    end = time.time() - start
    print("model training time", end)
    wandb.log({"training_time": end})

    # Close your wandb run
    wandb.run.finish()

    tf.keras.models.save_model(model, "model/keras_multi/")

In the code below, we set up the necessary parameters. We are only running two epochs to save time. But once you've got this working, you'll have all the information you need to build and run bigger TensorFlow models on Saturn Cloud. A single GPU processes all our batches every epoch. All the model parameters, as well as some extra elements like Notes and Tags, are tracked by Weights & Biases.

In [ ]:
model_params = {
    "n_epochs": 2,
    "base_lr": 0.02,
    "batchsize": 64,
    "classes": 285,
    "scale_batch": True,
}

wbargs = {
    **model_params,
    "Notes": "tf_v100_8x",
    "Tags": ["multi", "gpu", "tensorflow"],
    "dataset": "Birds",
    "architecture": "ResNet50",
}

Now run the model training process and save your trained model object to the Jupyter instance memory. 

In [ ]:
tester_plain = train_multigpu(wbargs=wbargs, **model_params)